In [1]:
# Name: Patrick Ijieh

# Reinforcement Learning

This is the 5th assignment for CAP 4630 and we will train an AI-based explorer to play a game by reinforcement learing. As domestrated below, in this game, the treasure (denoted by T) is on the right-most and the explorer (denoted by o) will learn to get the treasure by moving left and right. The explorer will be rewarded when it gets the treasure.  After serveral epoches, the explorer will learn how to get the treasure faster and finally it will go to the treasure by moving to right directly. \

Episode 1, Step1: o----T   \
... \
Episode 1, Step6: ---o-T   \
... \
Episode 1, Step10: -o---T \
... \
Episode 1, Step15: ----oT (finished) \

You will use **"Tasks"** and **"Hints"** to finish the work. **(Total 100 Points)** \

**Task Overview:**
- Train the explorer getting the treasure quickly through Q-learning method

## 1 Achieve Q-learning method ##
### 1.1 Model Preparation**(5 Points)**

Import useful packages and prepare hyperpaprameters for Q-learning methods.

**Tasks:**
1. Import numpy and rename it to np.
2. Import pandas and rename it to pd.
3. Import the library "time"
4. Set the parameter as suggested

**Hints:**
1. For your first trial, you may set as it is
2. You may explore other possibilities here when you complete the whole homework

In [2]:
#import packages here
import numpy as np
import pandas as pd
import time
import random

NUM_STATES = 6                # the width of 1-dim world
ACTIONS = ['left', 'right']   # the available actions to use
EPSILON = 0.9                 # the degree of greedy (0＜ε＜1)
ALPHA = 0.1                   # learning rate (0＜α≤1)
GAMMA = 0.9                   # discount factor (0＜γ＜1)
MAX_EPOCHS = 13               # the max epoches
FRESH_TIME = 0.3              # the interval time

### 1.2 Q table**(10 Points)**

Q table is a [states * actions] matrix, which stores Q-value of taking one action in that specific state. For example, the following Q table means in state s3, it is more likely to choose a1 because it's Q-value is 5.31 which is higher than Q-value 2.33 for a0 in s3(refer to Lecture slides 16, page 35).
![](https://drive.google.com/uc?export=view&id=1WGh7NYyYw6ccrxbDVdfbJmb_IhBfUyFf)

**Tasks:**
1. define the build_q_table function
2. **Print Out** defined Q-table

**Hints:**
1. Using pd.DataFrame to define the Q-table.(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
2. Initialize the Q-table with all zeros.

In [3]:
# define the function here
# def build_q_table(n_states, actions): #add your code here

def build_q_table(n_states: int, actions: list[str]) -> pd.DataFrame:
  arr = []
  for i in range(n_states):
    temp = []
    for j in actions:
      temp.append(0.0)
    arr.append(temp)


  qt = pd.DataFrame(arr, columns=actions, dtype=np.float32)
  return qt

q_table = build_q_table(NUM_STATES, ACTIONS)
print(q_table)

   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.0


### 1.3 Define action**(15 Points)**

In this section, we are going to define how an actor picks the actions. We introduce ε-greedy (In lecture slide 16, page 35). In the initial exploring stage, the explorer knows little about the environment. Therefore, it is better to explore randomly instead of greedy. ε-greedy is the value to control the degree of greedy. It can be changed with time lapsing. In this homework, we set it as fixed value EPSILON = 0.9. You can change it to explore the final effect.

**Tasks:**
1. define the choose_action function
2. **Print Out** sample action.

**Hints:**
1. You need to define two patterns: 1) non-greedy (i.e., random); 2) greedy.
2. Non-greedy should occupy (1-ε) senario while greedy should occupy ε senario. In this case, it means Non-greedy occupys 10% senario while greedy occupys 90% senario. (you could implement it by comparing a random number ranging from 0 to 1 with ε)
3. In the non-greedy pattern, the actor should choose the actions randomly.
4. In the greedy pattern, the actor should choose the higher Q-value action.
5. Don't forget the initial state which means all Q-value are zero and actor cannot choose greedily. You can treat it as non-greedy pattern.

In [4]:
# define the function here

# def choose_action(state, q_table):
#       # pick all actions from this state
#     if ... or ...:  # non-greedy or non-explored
#         ...
#     else:
#         ...    # greedy
#     return action_name

def state_is_zeroes(state: int, q_table: pd.DataFrame) -> bool:
  for action in ACTIONS:
    if q_table.loc[state, action] != 0:
      return False

  return True

# Given state and Q-table, choose action
def choose_action(state: int, q_table: pd.DataFrame) -> str:
  p = random.random()
  if p <= (1 - EPSILON) or state_is_zeroes(state, q_table):
    idx = random.randint(0, len(ACTIONS)-1)
    return ACTIONS[idx]

  else:
    max = -1
    action = ""
    for current_action in ACTIONS:
      if q_table.loc[state, current_action] > max:
        max = q_table.loc[state, current_action]
        action = current_action
    return action


sample_action = choose_action(0, q_table)
print(sample_action)

left


### 1.4 Interact with the environment**(30 Points)**

In this section, we need to give a feedback for our previous action, which means getting reward (R) for next state (S_next) based on current state (S_current) and action (A). In this problem, we get reward R=1 if we move to the treasure T spot, otherwise, we get R=0.

**Tasks:**
1. define get_env_feedback function
**Hints:**
1. This function contains two parameters S_current and A(ction), and return S_next and R(eward).
2. You need to consider two different senarios: 1) A = right; 2) A = left.
3. In the above two senarios, you need to consider the boundary, next state and rewards.
4. The update_env function is given to show changes for different steps in different episodes.
5. The validation for S_current and Action is shown below.

- S_current=0, sample_action = 'right', sample_feedback=(1,0)
- S_current=3, sample_action = 'right', sample_feedback=(4,0)
- S_current=4, sample_action = 'right', sample_feedback=('terminal', 1)
- S_current=0, sample_action = 'left', sample_feedback=(0,0)
- S_current=3, sample_action = 'left', sample_feedback=(2,0)
- S_current=4, sample_action = 'left', sample_feedback=(3, 0)

In [5]:
# define the function here
# def get_env_feedback(S_current, A):
#     # This is how agent will interact with the environment
#     if A == 'right':    # move right
#         ...
#     else:   # move left
#         ...
#     return S_next, R

def get_env_feedback(s_current: int, A: str) -> tuple[int | str, int]:
  R = 0
  s_next = -1

  if A == 'right':
    s_next = s_current + 1
    if s_next == NUM_STATES - 1:
      s_next = 'terminal'
      R = 1

  else:
    s_next = s_current - 1
    if s_next < 0:
      s_next = 0

  return s_next, R


S_current = 4
sample_action = 'right'
sample_feedback = get_env_feedback(S_current, sample_action)
print(sample_feedback)

('terminal', 1)


In [6]:
def update_env(S: int, episode: int, step_counter: int) -> None:
    # This is how environment be updated
    env_list = ['-']*(NUM_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = '  Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('{}\n'.format(interaction), end='')
        time.sleep(2)
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

### 1.5 Start Q-learning with defined functions**(40 Points)**

In this section, we are going to utilize all the functions defined above to do q-learning based on the optimal policy.
![](https://drive.google.com/uc?export=view&id=10ra6mLlBHlhGNTYWwdGANoa6lC1K_7at)

**Tasks**:
1. define reinforce_learning function

**Hints**:
1. You should write this function with loops to keep updating q-table until you get to the reward spot.
2. We have two loops, one is for different episodes and another one is for steps
3. Whenever we take a step to the reward spot, we should end the loop and start another episode.
4. Here is one possible example.

![](https://drive.google.com/uc?export=view&id=1oo-gk710XVXbbeI7AI0uZInrnKtqGqn7)

In [9]:
# define the function here
# def reinforce_learning():
#     # main part of RL loop
#     # build Q-table here
#     ...
#     #start training loop
#     for episode in range(MAX_EPOCHES):
#         step_counter = 0  #counter for counting steps to reach the treasure
#         S_current = ...     #start from S_current
#         is_terminated = ...   #flag to conrinue or stop the loop
#         update_env(S_current, episode, step_counter)   #update environment
#         while not is_terminated:
#             ...#choose one action
#             ...# take action & get next state and reward
#             ...#update Q-table
#             if S_next != 'terminal':                   #if the explorer doesn't get to the treasure
#                 q_target = ...   # next state is not terminal
#             else:
#                 q_target = ...     # next state is terminal
#                 is_terminated = ...    # terminate this episode

#             q_table.loc[S_current, A] = ...  # update Q-table
#             ...  # move to next state

#             update_env(S_current, episode, step_counter+1)
#             step_counter += 1
#     return q_table

def reinforce_learning() -> pd.DataFrame:
  q_t = build_q_table(NUM_STATES, ACTIONS)
  for episode in range(MAX_EPOCHS):
    step_counter = 0
    s_current = 0
    is_terminated = False
    update_env(s_current, episode, step_counter)
    while not is_terminated:
      q_target = 0.0
      A = choose_action(s_current, q_t)
      s_next, R = get_env_feedback(s_current, A)

      if s_next != 'terminal':
        q_target = ALPHA * (R + GAMMA * max_q_val(s_next, q_t) - q_t.loc[s_current, A])
      else:
        q_target = ALPHA * (R + GAMMA * 1 - q_t.loc[s_current, A])
        is_terminated = True

      q_t.loc[s_current, A] = q_t.loc[s_current, A] + q_target
      s_current = s_next
      step_counter += 1
      update_env(s_current, episode, step_counter)

  return q_t


def max_q_val(state: int, q_t: pd.DataFrame) -> int:
  rewards = []
  for action in ACTIONS:
    temp_reward = q_t.loc[state, action]
    rewards.append(temp_reward)
  return max(rewards)

In [8]:
# main function to run
if __name__ == "__main__":
    q_table = reinforce_learning()
    print('\r\nQ-table:\n')
    print(q_table)

----oT  Episode 1: total_steps = 9
----oT  Episode 2: total_steps = 38
----oT  Episode 3: total_steps = 48
----oT  Episode 4: total_steps = 10
----oT  Episode 5: total_steps = 5
----oT  Episode 6: total_steps = 5
----oT  Episode 7: total_steps = 9
----oT  Episode 8: total_steps = 7
----oT  Episode 9: total_steps = 7
----oT  Episode 10: total_steps = 7
----oT  Episode 11: total_steps = 5
----oT  Episode 12: total_steps = 5
----oT  Episode 13: total_steps = 5

Q-table:

       left     right
0  0.000000  0.011910
1  0.000119  0.064497
2  0.000000  0.243709
3  0.025610  0.647500
4  0.000000  1.417045
5  0.000000  0.000000
